<a href="https://colab.research.google.com/github/just12puja/Smart_Escapism_AI_Driven_Decoy_Deployment_Program/blob/main/final_deploy_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install -q streamlit pyngrok scikit-learn pandas matplotlib
print("✅ Dependencies installed.")

# Kill any existing Streamlit process that might be running on port 8501
import os
os.system("kill -9 $(lsof -t -i:8501)")
print("✅ Port 8501 cleared.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.4 MB/s eta 0:00:00
✅ Dependencies installed.
✅ Port 8501 cleared.


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import matplotlib.pyplot as plt

# --- 1. DATA GENERATION (Replicating the original Tkinter logic) ---
grid_size = 10
n_samples = 1000
data = []

for _ in range(n_samples):
    traffic = random.randint(100, 1000)
    anomaly = random.uniform(0, 1)
    alerts = random.randint(0, 20)

    # Simplified threat and heatmap generation for synthetic data
    threat_x, threat_y = random.randint(0, grid_size-1), random.randint(0, grid_size-1)
    # Replicating the heatmap score logic
    heatmap_score = int(min(10, anomaly*10 + alerts/2 + traffic/200))
    # Replicating the deploy decoy logic
    deploy_decoy = 1 if heatmap_score > 6 or (anomaly > 0.6 and alerts > 10) else 0

    data.append([traffic, anomaly, alerts, threat_x, threat_y, heatmap_score, deploy_decoy])

# Create the DataFrame and save it as 'historical_attacks.csv'
df_historic = pd.DataFrame(data, columns=["traffic", "anomaly", "alerts", "threat_x", "threat_y", "heatmap", "deploy"])
df_historic.to_csv("historical_attacks.csv", index=False)
print("✅ historical_attacks.csv created.")

# --- 2. TRAIN MODELS (Replicating the original Tkinter logic) ---

# Features
X = df_historic[["traffic", "anomaly", "alerts"]]

clf_x = RandomForestClassifier(n_estimators=100, random_state=42).fit(X, df_historic["threat_x"])
clf_y = RandomForestClassifier(n_estimators=100, random_state=42).fit(X, df_historic["threat_y"])
reg_heatmap = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, df_historic["heatmap"])
clf_deploy = RandomForestClassifier(n_estimators=100, random_state=42).fit(
    df_historic[["traffic","anomaly","alerts","heatmap"]], df_historic["deploy"]
)
print("✅ Random Forest Models trained.")

# --- 3. Decoy Predictor Function (Required by Streamlit app) ---
def predict_decoy_location(asset, threat):
    Ax, Ay = asset
    Tx, Ty = threat
    Dx = (Ax + Tx) // 2 # Use floor division
    Dy = (Ay + Ty) // 2
    return (Dx, Dy)

✅ historical_attacks.csv created.
✅ Random Forest Models trained.


In [3]:
from pyngrok import ngrok
from getpass import getpass

# Get the ngrok authentication token securely
print("Please enter your ngrok Authtoken. You can get it from your ngrok dashboard:")
ngrok_token = getpass()

# Set the ngrok authentication token
try:
    ngrok.set_auth_token(ngrok_token)
    print("✅ Ngrok Authtoken set successfully.")
except Exception as e:
    print(f"❌ Error setting ngrok Authtoken: {e}")

# This cell does not produce a visible output after input, but it configures ngrok for the next steps.

Please enter your ngrok Authtoken. You can get it from your ngrok dashboard:
··········
✅ Ngrok Authtoken set successfully.


In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import random
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import matplotlib.pyplot as plt

# --- CONFIG ---
grid_size = 10
asset = (2, 2)
CSV_FILE = "historical_attacks.csv"

# --- 1. LOAD DATA & TRAIN MODELS (Cached for speed) ---
@st.cache_resource
def load_and_train_models(file_path):
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        st.error(f"Error: File {file_path} not found. Rerun the previous cells.")
        st.stop()

    X = df[["traffic", "anomaly", "alerts"]]

    clf_x = RandomForestClassifier(n_estimators=100, random_state=42).fit(X, df["threat_x"])
    clf_y = RandomForestClassifier(n_estimators=100, random_state=42).fit(X, df["threat_y"])
    reg_heatmap = RandomForestRegressor(n_estimators=100, random_state=42).fit(X, df["heatmap"])
    clf_deploy = RandomForestClassifier(n_estimators=100, random_state=42).fit(
        df[["traffic","anomaly","alerts","heatmap"]], df["deploy"]
    )
    return clf_x, clf_y, reg_heatmap, clf_deploy

clf_x, clf_y, reg_heatmap, clf_deploy = load_and_train_models(CSV_FILE)

# --- 2. DECOY PREDICTOR FUNCTION ---
def predict_decoy_location(asset, threat):
    Ax, Ay = asset
    Tx, Ty = threat
    Dx = (Ax + Tx) // 2
    Dy = (Ay + Ty) // 2
    return (Dx, Dy)

# --- 3. STREAMLIT APP LAYOUT & LOGIC ---
st.set_page_config(layout="wide")
st.title("🛡️ Real-Time Decoy Deployment Predictor")
st.caption(f"Simulation running with Random Forest models trained on {CSV_FILE} data.")

if 'log_df' not in st.session_state:
    st.session_state.log_df = pd.DataFrame(columns=[
        "Cycle", "Traffic", "Anomaly", "Alerts", "Threat", "Heatmap", "Decoy"
    ])
if 'cycle' not in st.session_state:
    st.session_state.cycle = 0

col1, col2 = st.columns([1, 1])

def run_simulation_step(cycle):
    traffic = random.randint(100, 1000)
    anomaly = round(random.uniform(0, 1), 2)
    alerts = random.randint(0, 20)

    features = np.array([[traffic, anomaly, alerts]])
    pred_x = clf_x.predict(features)[0]
    pred_y = clf_y.predict(features)[0]
    pred_heatmap = reg_heatmap.predict(features)[0]

    deploy_pred = clf_deploy.predict([[traffic, anomaly, alerts, pred_heatmap]])[0]

    decoy_loc = predict_decoy_location(asset, (pred_x, pred_y)) if deploy_pred == 1 else None

    return traffic, anomaly, alerts, pred_x, pred_y, pred_heatmap, decoy_loc

def update_dashboard():
    st.session_state.cycle += 1
    cycle = st.session_state.cycle

    traffic, anomaly, alerts, pred_x, pred_y, pred_heatmap, decoy_loc = run_simulation_step(cycle)

    result_text = f"""
    **Traffic:** {traffic} | **Anomaly:** {anomaly} | **Alerts:** {alerts}
    **Predicted Threat Location:** ({int(pred_x)}, {int(pred_y)}) | **Predicted Heatmap Score:** {pred_heatmap:.2f}
    **Decoy Deployment:** {'✅ YES at ' + str(decoy_loc) if decoy_loc else '❌ NO'}
    """

    with col1:
        st.subheader(f"Cycle {cycle} Live Status")
        st.markdown(result_text)

        heatmap = np.zeros((grid_size, grid_size))
        heatmap[int(pred_y), int(pred_x)] = pred_heatmap

        fig, ax = plt.subplots(figsize=(6, 5))
        im = ax.imshow(heatmap, cmap="hot", interpolation="nearest", origin='lower')

        ax.scatter(asset[0], asset[1], c="lime", marker="s", s=200, label="Asset (2,2)")
        ax.scatter(pred_x, pred_y, c="red", marker="X", s=200, label="Threat")
        if decoy_loc:
            ax.scatter(decoy_loc[0], decoy_loc[1], c="cyan", marker="o", s=200, label="Decoy")

        ax.set_title("Predicted Threat & Decoy Deployment Map")
        ax.set_xticks(np.arange(grid_size))
        ax.set_yticks(np.arange(grid_size))
        ax.legend(loc="upper right")
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

        st.pyplot(fig)
        plt.close(fig)

    new_entry = pd.DataFrame([{
        "Cycle": cycle,
        "Traffic": traffic,
        "Anomaly": anomaly,
        "Alerts": alerts,
        "Threat": f"({int(pred_x)},{int(pred_y)})",
        "Heatmap": f"{pred_heatmap:.2f}",
        "Decoy": f"{decoy_loc}" if decoy_loc else "No"
    }])

    st.session_state.log_df = pd.concat([new_entry, st.session_state.log_df], ignore_index=True)

    with col2:
        st.subheader("Security Log (Last 10 Cycles)")
        st.dataframe(st.session_state.log_df.head(10), use_container_width=True)

with st.sidebar:
    st.header("Controls")
    if st.button("Start/Restart Simulation", type="primary"):
        st.session_state.cycle = 0
        st.session_state.log_df = pd.DataFrame(columns=[
            "Cycle", "Traffic", "Anomaly", "Alerts", "Threat", "Heatmap", "Decoy"
        ])
        st.rerun()

    live_update = st.checkbox("Enable Live Update (Every 2s)", value=True)

    if st.session_state.cycle > 0:
        st.markdown(f"**Current Cycle:** {st.session_state.cycle}")

if st.session_state.cycle == 0 or live_update:
    update_dashboard()
    if live_update:
        time.sleep(2)
        st.rerun()

Writing app.py


In [5]:
from pyngrok import ngrok
import subprocess
import time

# Set up ngrok tunnel using the previously configured token
try:
    http_tunnel = ngrok.connect(8501, bind_tls=True)
    print(f"🌍 Streamlit Public URL: {http_tunnel.public_url}")
except Exception as e:
    # This might happen if the token was wrong, or if ngrok itself has connection issues
    print(f"❌ Failed to start ngrok tunnel: {e}")
    print("If the Streamlit server starts, you might be able to find the URL in the console output below.")

# Run the Streamlit app in the background
subprocess.Popen(["streamlit", "run", "app.py", "--server.enableCORS=false", "--server.enableXsrfProtection=false"],
                 stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("\n🚀 Streamlit is starting... Wait a few seconds and click the URL above.")

🌍 Streamlit Public URL: https://e638484ba4f9.ngrok-free.app

🚀 Streamlit is starting... Wait a few seconds and click the URL above.
